In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Next-Generation Astronomical Transient Classification via Large Language Models

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/turanbulmus/spacehack/blob/main/01%20-%20Prompt%20Engineering.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fturanbulmus%2Fspacehack%2Fmain%2F01%20-%20Prompt%20Engineering.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/turanbulmus/spacekhack/main/01%20-%20Prompt%20Engineering.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/turanbulmus/spacehack/blob/main/01%20-%20Prompt%20Engineering.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| Authors |\
| Turan Bulmus | https://github.com/turanbulmus | \
| Fiorenzo Stoppa | https://github.com/FiorenSt |


### Set Google Cloud environment variables information and initialize Clients

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).
To get started using Vertex AI, you must have an existing Google Cloud project and the following:
* [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). 
* [Enable the Big Query API](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com).

In [2]:
PROJECT_ID = "<YOUR_PROJECT_ID>"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
EXPERIMENT_NAME = "supernovadetection" # @param {type:"string"}
# Make sure that dataset is created in Big Query
DATASET_ID = "spacehack" # @param {type:"string"}
import vertexai
from google.cloud import bigquery


vertexai.init(project=PROJECT_ID, location=LOCATION, experiment=EXPERIMENT_NAME)
bq_client = bigquery.Client(project=PROJECT_ID)

/home/user/spacehack/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


/home/user/spacehack/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


## Import libraries and define helper functions

In [3]:
import gdown 
import base64
import json
import random, os
import time, datetime
from pathlib import Path

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from vertexai.generative_models import GenerativeModel, Part, FinishReason, Image
from google.cloud import bigquery
import google.cloud.aiplatform as aiplatform

from helper_functions import build_run_batch, if_tbl_exists, create_ex, save_picture, save_prompt, build_experiment_vars

In [ ]:
PERSONA = """<PERSONA>
You are an experienced astrophysicist, and your task is to classify astronomical transients into Real or Bogus based on a given set of 3 images. You have seen thousands of astronomical images during your lifetime and you are very good at making this classification by looking at the images and following the instructions.
</PERSONA>"""

TASK = """<TASK>
Your task is to read the INSTRUCTIONS, look at the 3 images (New, Reference and Difference images) and classify if the source at the centre of the cutout and inside the red circle is a Real or Bogus astronomical transient. Provide your thought process to explain how you reasoned to provide the response. Respond in json format
</TASK>\n
"""

INSTRUCTIONS = """\n<INSTRUCTIONS>
**1. Purpose**
Help vet astronomical data for the Real/Bogus classification. The goal is for you to use your expertise to distinguish between real and bogus sources. 

**2. Information Provided**
You will be shown three astronomical image cutouts:
a) **New Image:** The newest image centered at the location of the suspected transient source. 
b) **Reference Image:** A reference image from the same telescope of the same part of the sky to be used for comparison. It shows if the source was already there in the past or not.
c) **Difference Image:** The residual image after the new and reference images are subtracted. Real sources should appear in this cutout as circular objects with only positive (white pixels) or only negative (black pixels) flux. 

**3. Criteria for Classification**
- **Real Source:** 
  - **Shape:** Circular shape at the center of the cutout with a visual extent of ~5-10 pixels, varying with focus conditions.
  - **Brightness:** Positive flux (white pixels) in either the new or reference image. Positive or negative flux in the Difference image. 
  - **Variability:** The source at the center can fade or brighten between the new and reference images, appearing as positive or negative in the Difference image.
  - **Presence:** The source may (dis)appear between the new and reference images. A source may also appear on top of an underlying source (e.g., supernova on a galaxy).

- **Bogus Source:** 
  - **Shape:** Non-circular shape (e.g., elongated). This includes irregular shapes, positive or negative, like streaks or lines caused by cosmic-rays, diffraction spikes and cross-talk.
  - **Brightness:** Negative flux (black pixels) at the center of the cutout in either the new or reference image. The source at the center can never be negative in the New or Reference image, only in the Differnece.
  - **Misalignment:** If the source in the New and Reference images is misaligned, it will show a Yin-Yang pattern (both white and black) in the Difference image.

**4. Additional Guidance** 
- **Contextual Information:** Focus on the source at the center of the cutouts inside the red circle, but consider nearby sources to diagnose potential problems.  
- **Examples:** Refer to provided visual examples of real and bogus sources to aid in identification. 
- **Judgment Criteria:** For ambiguous cases or borderline scenarios, consider the overall context and consistency with known characteristics of real and bogus sources.

</INSTRUCTIONS>"""


METHOD = """<METHOD>
1. **Focus on the Red Circle**: Start by examining the source located at the center of the cutout and inside the red circle. The images are prepared so that the source of interest is clearly marked for you to analyze.

2. **Analyze Each Image Individually**:
   - **New Image**: Check for the presence, shape, and brightness of the source in the new image.
   - **Reference Image**: Compare the source's properties in the reference image to those in the new image.
   - **Difference Image**: Observe the residuals that result from subtracting the reference image from the new image. Look for patterns (circular, positive/negative flux) that match characteristics of Real or Bogus sources.

3. **Evaluate Features**:
   - Examine the shape, brightness, and other relevant features (e.g., artifacts, misalignments) of the source in each image.
   - Determine if these features are consistent with a Real or Bogus classification based on the criteria provided in the instructions.

4. **Consider Relationships Between Images**:
   - Compare the new, reference, and difference images to understand any changes in the source over time.
   - Look for discrepancies or confirmations that might support or contradict a particular classification.

5. **Employ a Chain-of-Thought Reasoning**:
   - Clearly outline each observation you make and explain how it contributes to your decision-making process.
   - If you find any contradictions or ambiguous features, acknowledge them and provide reasoning for your final decision.

6. **Assign an Interest Score**:
   - After determining if the source is Real or Bogus, assign an appropriate interest score:
     - 'No interest' for Bogus sources.
     - 'Low interest' for variable transients.
     - 'High interest' for explosive transients.

7. **Prepare the Final Output in JSON Format**:
   - Format your response as a JSON object containing:
     - The classification ('Real' or 'Bogus').
     - An explanation detailing your thought process and observations.
     - The assigned interest score.

8. **Example Output**:
   - Refer to the provided examples to see the expected format and detail level of your response.
</METHOD>
"""

# Collapse the System Instructions into a single variable
stat_prompt = PERSONA + TASK + INSTRUCTIONS + METHOD

In [ ]:
import pandas as pd
import numpy as np
import os

# Load the dataset
file_path_data = '/home/user/spacehack/MeerLICHT_dataset/MeerLICHT_images.npy'
file_path_labels_csv = '/home/user/spacehack/MeerLICHT_dataset/MeerLICHT_labels.csv'

triplets = np.load(file_path_data)

# Load labels from CSV file
labels_df = pd.read_csv(file_path_labels_csv)

# Upload Labels to BigQuery
labels_id = "MeerLICHT_labels_df"
labels_ref = bq_client.dataset(DATASET_ID).table(labels_id)
create_table_flag = if_tbl_exists(bq_client, labels_ref)

if create_table_flag != True:
    bq_client.load_table_from_dataframe(labels_df, labels_ref)

# Sample indexes for saving pictures
sample_indexes = [0, 1, 3, 4, 8, 48, 77, 1179, 1180, 1181, 1191, 1193, 592, 685, 3216, 54, 3004, 593, 1035, 290, 2584]

# Save example pictures
for i in sample_indexes:
    save_picture(triplets, i, True)

# Identify valid indices by removing any with corrupt data
valid_indices = np.where(~np.isnan(triplets).any(axis=(1, 2, 3)))[0]

# Exclude sample_indexes from the prediction set
batch_index = np.setdiff1d(valid_indices, sample_indexes)

# Save prediction pictures
for t in batch_index:
    save_picture(triplets, t, False)

print(f"Saved {len(batch_index)} prediction pictures excluding examples.")

In [ ]:
## DESCRIPTION INDEX 0:
desc1 = {
  "class": "Bogus",
  "explanation": "In the New image, a diffraction spike is observed near the center. The Reference image also shows a diffraction spike at the same location. In the Difference image, a negative residual of the bright diffraction spike from the Reference image is clearly visible. The consistent presence of diffraction spikes in all three images, without a clear circular source, confirms that this is a bogus source.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 1:
desc2 = {
  "class": "Bogus",
  "explanation": "In the New image, a negative elongated artifact is present at the center. The Reference image does not show any source at the same location. In the Difference image, the same negative artifact appears, which results from the negative clump of pixels in the New image. Since a real source cannot be negative in the New image, this is classified as a bogus source.",
  "interest_score": "No interest"
}


## DESCRIPTION INDEX 3:
desc3 = {
  "class": "Bogus",
  "explanation": "In the New image, the source appears as a streak of several bright pixels and is not circular. The Reference image shows no source at the same location. The Difference image shows the same streak of pixels as in the New image. The sharp, streak-like appearance in the New image suggests that this is most likely a cosmic ray rather than a real source.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 4:
desc4 = {
  "class": "Bogus",
  "explanation": "The New image does not have any source at the centre of the cutout. The Reference image shows a source appearing as a streak of a few bright pixels, which is not circular. The difference image shows the negative residual of the same streak present in the Reference image. This is too sharp to be a real source and is likely a cosmic ray that was not flagged during the creation of the Reference image.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 8:
desc5= {
  "class": "Bogus",
  "explanation": "No source is present in the New image. In the Reference image, a source appears as a negative circular object. The Difference image presents a faint positive residual of the source in the Reference image. A source cannot be negative in the Reference image, this is not a real source.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 48:
desc6 = {
  "class": "Bogus",
  "explanation": "The New image does not have any source at the centre of the cutout. In the Reference image, the source appears very elongated. The Difference shows the same negative elongated source, supporting the conclusion that it a bogus source.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 77:
desc7 = {
  "class": "Bogus",
  "explanation": "In the New image, a small elongated source is visible, surrounded by several other sources. The Reference image shows no source at the same location, but it does show all the other sources. In the Difference image, the residual is positive but its elongation confirms this is a bogus source.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 1179:
desc8 = {
  "class": "Real",
  "explanation": "The New image shows a source at the center. The Reference image also shows the same source in the same location. The Difference image has a positive residual, indicating the source has brightened. This pattern suggests the source is a real variable star.",
  "interest_score": "Low interest"
}


## DESCRIPTION INDEX 1180:
desc9 = {
  "class": "Real",
  "explanation": "The New image shows a source at the center. The Reference image also shows the same source in the same location. The Difference image has a negative residual, indicating the source has dimmed. This pattern suggests the source is a real variable star.",
  "interest_score": "Low interest"
}


## DESCRIPTION INDEX 1181:
desc10 = {
  "class": "Real",
  "explanation": "The New image shows no source at the center. The Reference image shows a circular source in the same location. The Difference image displays a negative circular residual, consistent with a transient that has disappeared.",
  "interest_score": "High interest"
}


## DESCRIPTION INDEX 1191:
desc11 = {
  "class": "Real",
  "explanation": "The New image shows a bright circular source at the center. The Reference image shows no source in the same location. The Difference image displays a positive circular residual, indicating a real explosive transient.",
  "interest_score": "High interest"
}

## DESCRIPTION INDEX 1193:
desc12 = {
  "class": "Real",
  "explanation": "The New image shows a source at the center. The Reference image also shows the same source in the same location. The Difference image displays a positive residual, indicating the source has brightened. A cosmic ray artifact is visible to the left, but the central source is unaffected and remains a valid transient.",
  "interest_score": "Low interest"
}

## DESCRIPTION INDEX 592:
desc13 = {
  "class": "Bogus",
  "explanation": "The New image shows a diffuse source at the center, aligned with a 45-degree diffraction spike from a bright source at the corner of the cutout. The Reference image also shows a diffraction spike and a similar blob. The Difference image displays a positive blob, indicating it is an artifact caused by the diffraction spike, which can produce blobs or irregular shapes.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 685:
desc14 = {
  "class": "Bogus",
  "explanation": "The New image shows no source at the center. The Reference image shows a faint positive trail cutting diagonally across the image with a circular source at the center, likely caused by a blinking object like an airplane or satellite. The Difference image displays both the trail and a negative blob, confirming the source is a non-astronomical artifact.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 3216:
desc15 = {
  "class": "Real",
  "explanation": "The New image shows a source at the center, superimposed on a diffuse galaxy. The Reference image displays the galaxy but no source at the same location. The Difference image reveals a faint, positive circular feature, consistent with a supernova emerging within the galaxy.",
  "interest_score": "High interest"
}

######################################################################## NEW IMAGES AND EXPLANATIONS ########################################################################

## DESCRIPTION INDEX 54:
desc16 = {
  "class": "Bogus",
  "explanation": "The New image shows a quite bright trail that cuts the entire image diagonally with no obvious brighter region of the train in the center. The Reference image shows a similar trail cutting diagonally across the image. The Difference image shows a faint postive residual of the trail. This is a bogus source, likely a diffraction spike since it was present in both New and Reference images.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 3004:
desc17 = {
  "class": "Bogus",
  "explanation": "The New image shows a diffuse source that is however not centred in the red circle, so not of interest. The Reference image shows a negative source, darker than the background and therefore an artifact. The Difference image shows a faint positive source, but it is caused by the negative artifact in the reference image, this is a bogus source.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 593:
desc18 = {
  "class": "Bogus",
  "explanation": "The New image shows a source darker than the background, and therefore an artifact. The Reference image shows both a postive source and a negative one, both slightly off centre. The Difference image shows a negative source, but it is caused by the artifacts present in both the New and Reference images.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 1035:
desc19 = {
  "class": "Bogus",
  "explanation": "The New image shows no obvious source at the center. The Reference image has a diffused source at the centre, that however is elongated and clearly aligned with a 45-degree diffraction spike from a bright source at the bottom right corner of the image. The Difference image shows a really diffuse negative blob, but this is an artifact caused by the diffraction spike.",
  "interest_score": "No interest"
}

## DESCRIPTION INDEX 290:
desc20 = {
  "class": "Bogus",
  "explanation": "The New image shows a diffuse source and elongated source close to the CCD detector edge. The Reference image shows no source at the centre of the circle, but a similar artifact can be seen close to the edge of the detector that is slightly shifted in the Reference image. The Difference image shows a positive elongated source, but this is likely a bogus source due to the vicinity of the edge of the detector.",
  "interest_score": "No interest"
}


## DESCRIPTION INDEX 2584:
desc21 = {
  "class": "Bogus",
  "explanation": "The New image show a really small and bright source of a few pixels, liekly a cosmic ray. The Reference image does not show any soure at the centre, as expected from a cosmic ray happening only in the New image. The Difference image shows a somewhat odd positive residuals, further confriming this is a bogus source.",
  "interest_score": "No interest"
}


descriptions = [str(desc1), str(desc2), str(desc3), str(desc4), str(desc5), str(desc6),str(desc7), str(desc8), str(desc9), str(desc10), str(desc11), str(desc12), str(desc13), str(desc14), str(desc15), str(desc16), str(desc17), str(desc18), str(desc19), str(desc20), str(desc21)]

### Write the examples used in a readable format to be saved as a txt file for tracebility
example_description = list(zip(["DESCRIPTION INDEX: " + str(x) for x in sample_indexes], descriptions))

In [ ]:
# Report 21 examples for the dynamic prompt
EXAMPLES = ["<EXAMPLES>\n"]
for i in range(len(sample_indexes)):
    
    str_EX = f"""Example {i+1}:
    """
    all_list = create_ex(sample_indexes[i], True)
    all_list.insert(0, str_EX)
    all_list.append(descriptions[i])
    all
    for k in all_list:
        EXAMPLES.append(k)
EXAMPLES.append("\n</EXAMPLES>\n")

In [ ]:
# Start logging the experiment

## Prepare the variables
timestamp = datetime.datetime.now()
formatted_datetime = timestamp.strftime('%Y%m%d%H%M')


## Log the experiments variables
### Create the run name with timestamp
run_name = "run" + formatted_datetime
DESCRIPTION = """Changed the instructions to give more structure.
The hyperparameters are: 
Temperature: 1
Top P: 0.5
""" # @param {type:"string"}
MODEL = "gemini-1.5-pro-002" # @param [gemini-1.5-pro-001", "gemini-1.5-flash-001", "gemini-1.0-pro-002"]
TEMPERATURE = 1 # @param {type:"slider", min:0, max:2, step:0.1}
TOP_P = 0.5 # @param {type:"slider", min:0, max:1, step:0.05}
PROMPT_FILE = save_prompt(stat_prompt + '\n'.join([a + "\n" + b + "\n" for (a,b) in example_description]), run_name)

# Build the experimentation variables
exp_vars = build_experiment_vars(description=DESCRIPTION,prompt=PROMPT_FILE, model=MODEL, temperature=TEMPERATURE, top_p=TOP_P)
# Start the run
aiplatform.start_run(run_name)
# Log the experiment variables
aiplatform.log_params(exp_vars)

In [ ]:
# Build and then the batch run requests
pred_df = build_run_batch(bq_client, batch_index, labels_ref, PROJECT_ID, DATASET_ID, run_name, MODEL, stat_prompt, EXAMPLES, TEMPERATURE, TOP_P)

In [10]:
download_query = f"""
  SELECT index_no, actual, predicted, explanation, interest_score
  FROM {PROJECT_ID}.spacehack.run202412191345"""
bq_client.query_and_wait(download_query)

In [ ]:
# Generate the confusion matrix for the results
plt.clf()
pred_cleaned = pred_df[(pred_df.predicted == "Real") | (pred_df.predicted == "Bogus")]
cm = confusion_matrix(pred_cleaned.actual, pred_cleaned.predicted)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()
# Calculate TP, TN, FP, FN
TP = cm[0][0]
TN = cm[1][1]
FP = cm[0][1]
FN = cm[1][0]

print(f"Total is {TP+TN+FP+FN}")

# Print the metrics
print(f"Accuracy is {(TN+TP)/(TN+TP+FP+FN)}")
print(f"Precision is {TP/(TP+FP)}")
print(f"Recall is {TP/(TP+FN)}")

In [ ]:
# Log the KPI and conclude the experiment
aiplatform.log_metrics(build_experiment_vars(accuracy=(TN+TP)/(TN+TP+FP+FN), precision=TP/(TP+FP), recall=TP/(TP+FN)))
aiplatform.end_run()